In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import requests
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
db_url = os.getenv("DB_URL")
api_url = os.getenv("API_URL")

# Fetch data from the API
def fetch_exchange_rates(api_url):
    response = requests.get(api_url)
    data = response.json()
    return data

# Process API response with Pandas
def process_api_response(data):
    df = pd.DataFrame(data['rates'].items(), columns=['currency', 'exchange_rate'])
    df['timestamp'] = datetime.now()
    return df

# Connect to PostgreSQL database
engine = create_engine(db_url)

# Fetch data, process, and insert into PostgreSQL table
exchange_rates_data = fetch_exchange_rates(api_url)
df = process_api_response(exchange_rates_data)

# Create a table if it doesn't exist
table_name = 'exchange_rates'
if not engine.dialect.has_table(engine, table_name):
    # Add columns for each currency based on the API response
    for currency in exchange_rates_data['rates']:
        df[currency] = None

    # Create the table in PostgreSQL
    df.head(0).to_sql(table_name, engine, if_exists='replace', index=False)

# Update the DataFrame with exchange rates for each currency
for currency in df['currency']:
    df[currency] = df['exchange_rate'] / exchange_rates_data['rates'][currency]

# Drop the temporary 'exchange_rate' column
df.drop(columns=['exchange_rate'], inplace=True)

# Insert data into PostgreSQL table
df.to_sql(table_name, engine, if_exists='append', index=False)


KeyError: 'rates'